In [2]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb, HighpassFilter
from pedalboard.io import AudioFile
import music21
from keyboard_layout import KeyboardLayout
from tone import Tone
from audio_player import AudioPlayer
from time_going import Timeline

smplRate = 44100

board = Pedalboard([Chorus(), Reverb(room_size=0.25),HighpassFilter(5000)])

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')


complete left to right loaded as keyboard layout successfully.


In [12]:
e = 2
harmonics = [(i,i/2) for i in range(2,20)]
tonemaker_1 = Tone(harmonics = harmonics, fade_duration = 0.06)

def scaling_1(point, base):
    return 40 * np.power(base,point/12)

def scaling_2(point):
    scale = [2,2,1,3,1,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

def final_scaling(x):
    return scaling_1(scaling_2(x),2)



In [4]:
timeline = Timeline(IS_LOOPED = False)

In [5]:
audioplayer = AudioPlayer(timeline,board)
audioplayer.start()

In [6]:
keyboard_layout.start(final_scaling, audioplayer, tonemaker_1, timeline)

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [14]:
import time
score = music21.corpus.parse('bach/bwv65.2.xml')
# score.show()
key = score.analyze('key')
tonicPitchClass = key.tonic.pitchClass
scoreChordify = score.chordify()
scoreChordsFlat = scoreChordify.flatten()
chords = scoreChordsFlat.getElementsByClass('Chord')

itr = 1.9
for chord in chords:
    tonemaker_1.harmonics = [(np.power(itr,np.log2(i)),np.power(itr,np.log2(i))) for i in range(2,20)]
    itr = itr + 0.005
    
    position = audioplayer.position

    for p in chord.pitches:
        fundamental = scaling_1(p.midi-30,itr)
        tone = tonemaker_1.make(fundamental)/10
        timeline.add(tone, position)
    time.sleep(chord.duration.quarterLength/2)
    
    # i.key = k


    # rootPitchClass = chord.root().pitchClass
    # diff = rootPitchClass - tonicPitchClass
    # if(diff < 0):
    #     diff += (11+1)
    # print(diff)


# print(s.metadata.all())

In [8]:
# closing audio in case of emergency
audioplayer.stop()

In [15]:
timeline.clear()

In [24]:
timeline.save()

mandarvordi saved as timeline successfully.


In [7]:
timeline.load()

mandarvordi loaded as timeline successfually.
